In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import joblib
import functools
import copy
import operator
import pickle
import spacy
import os
import re, string, unicodedata
import nltk
!pip install contractions
!pip install inflect
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

     |████████████████████████████████| 327kB 4.9MB/s 
     |████████████████████████████████| 266kB 7.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85398 sha256=4fa0beb70d22e6b5aaae3840408ead15b6a8199dd43042f5acbeaf9693df7b6c
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    text=re.sub('\n',' ',text)
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

def replace_contractions(text):
    return contractions.fix(text)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def get_wordnet_pos(tag):
    tag_dict = {"J": wordnet.ADJ,"N": wordnet.NOUN,"V": wordnet.VERB,"R": wordnet.ADV}
    return tag_dict.get(tag,wordnet.NOUN)

# def lemmatize(words):
#     """Lemmatize words in list of tokenized words"""
#     lemmatizer = WordNetLemmatizer()
#     lemmas = []
#     posTagged = nltk.pos_tag(words)
#     wordnetTagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1][0])), posTagged))
#     for word,tag in wordnetTagged:
#       lemma = lemmatizer.lemmatize(word,tag)
#       lemmas.append(lemma)
#     return lemmas

def lemmatizeSpacy(words):
  sent = ""
  lwords = []
  for word in words:
    sent += word + " " 
  model = spacy.load("en_core_web_sm")
  tokens = model(sent)
  # for i in range(len(tokens)):
  #   lwords.append((tokens[i].lemma_,i+1))
  for token in tokens:
    lwords.append(token.lemma_)
  return lwords

def preProcess_html(fileName):
    sample = denoise_text(fileName)
    sample = replace_contractions(sample)
    words = nltk.word_tokenize(sample)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = lemmatizeSpacy(words)
    words = remove_stopwords(words)
    # words = lemmatize(words)
    # words = lemmatizeSpacy(words)
    return words  

def clean_text(text):
    # text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z0-9A-Z]',' ',text)
    text=re.sub(' +',' ',text)
    return text

def preProcessSentence(fileName):
    sample = clean_text(fileName)
    sample = replace_contractions(sample)
    words = nltk.word_tokenize(sample)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatizeSpacy(words)    
    return words

In [4]:
def getDocNames(dt,ListdocId):
  Docnames = []
  for Sdocid in ListdocId:
    for docname, docid in dt.items():  
      if Sdocid == docid:
        Docnames.append(docname)
  return Docnames

def oneWord(Q,positional_index,document_index_map):
  if Q not in positional_index.keys():
    print("Terms Not Present")
    print("Number of documents retrieved: {0}".format(0))
    print("List of document names retrieved: --")
  else:
    p = positional_index[Q][1].keys()
    Listdocs = getDocNames(document_index_map,p)
    print("Number of documents retrieved: {0}".format(len(Listdocs)))
    print("List of document names retrieved: {0}".format(", ".join(Listdocs)))

def getPostings(terms,dt):
    #all terms in the list are guaranteed to be in the index
    return [ dt[term] for term in terms ]


def getDocsFromPostings(postings):
    #no empty list in postings
    return [ [x[0] for x in p] for p in postings ]

def intersectLists(lists):
    if len(lists)==0:
        return []
    lists.sort(key=len)
    return list(functools.reduce(lambda x,y: set(x)&set(y),lists))

def getDictionary(query,positional_index):
  d = {}
  for i in query:
    d[i] = [[x,positional_index[i][1][x]] for x in positional_index[i][1]]
  return d

def phraseQueries(Query,positional_index,document_index_map):
  for term in Query:
    if term not in positional_index.keys():
      print("Terms Not Found")
      print("Number of documents retrieved: {0}".format(0))
      print("List of document names retrieved: --")
      return
  dictionary = getDictionary(Query,positional_index)
  postings = getPostings(Query,dictionary)
  # print(postings)
  docs = getDocsFromPostings(postings)
  # print(docs)
  docs = intersectLists(docs) 
  # print(docs)
  for i in range(len(postings)):
    postings[i]=[x for x in postings[i] if x[0] in docs]
  # print(postings)
  postings=copy.deepcopy(postings)
  for i in range(len(postings)):
    for j in range(len(postings[i])):
      postings[i][j][1]=[x-i for x in postings[i][j][1]]

  # print(postings)
  result=[]
  for i in range(len(postings[0])):
    li=intersectLists([x[i][1] for x in postings])
    if li==[]:
      continue
    else:
      result.append(postings[0][i][0])    #append the docid to the result
  if len(result) == 0:
    print("Number of documents retrieved: {0}".format(0))
    print("List of document names retrieved: --")
  else:
    print("Number of documents retrieved: {0}".format(len(result)))
    print("List of document names retrieved: {0}".format(", ".join(getDocNames(document_index_map,result))))

In [6]:
if __name__== "__main__":
  positional_index = joblib.load('/content/drive/MyDrive/IR_ASSIGNMENT_1/without_StopWords/positional_indexes')
  document_index_map = joblib.load('/content/drive/MyDrive/IR_ASSIGNMENT_1/without_StopWords/document_index_map')
  document_index_map_keys = list(document_index_map.keys())
  document_index_map_values = list(document_index_map.values())
  number_of_queries = int(input("Enter number of queries:   "))
  # Queries = ["plum-coloured swelling","Telephones jangle and typewriters","good government","the mountains bordering","good and day","good day","rabbits, hares, partridges and skylarks","The terrified citizens flocked","Sherlock Holmes","the lion, the tiger","lion tiger","I Cried","British bark Sophy Anderson","Gfile Distribution Center","Abbey Grange, Marsham","Descent and Easy","AND ON THAT BRANCH THERE WAS A NEST"]
  for q in range(number_of_queries):
    query = str(input("Enter the query with terms separated by space:  "))
    print("Query : "+query)
    if(not (query and not query.isspace())):
      print("Empty Query")
    else:
      query = preProcessSentence(query)
      print("Query Terms: {0}".format(query))
      if len(query) == 1:
        oneWord(query[0],positional_index,document_index_map)
      else:
        phraseQueries(query,positional_index,document_index_map)
    
    print("\n")

Enter number of queries:   2
Enter the query with terms separated by space:  good day
Query : good day
Query Terms: ['good', 'day']
Number of documents retrieved: 22
List of document names retrieved: enchdup.hum, gulliver.txt, outcast.dos, mazarin.txt, melissa.txt, 13chil.txt, sick-kid.txt, brain.damage, fantasy.hum, fantasy.txt, breaks2.asc, bruce-p.txt, startrek.txt, superg1, aesop11.txt, aesopa10.txt, horswolf.txt, hound-b.txt, fic5, history5.txt, forgotte, srex.txt


Enter the query with terms separated by space:  good and day
Query : good and day
Query Terms: ['good', 'day']
Number of documents retrieved: 22
List of document names retrieved: enchdup.hum, gulliver.txt, outcast.dos, mazarin.txt, melissa.txt, 13chil.txt, sick-kid.txt, brain.damage, fantasy.hum, fantasy.txt, breaks2.asc, bruce-p.txt, startrek.txt, superg1, aesop11.txt, aesopa10.txt, horswolf.txt, hound-b.txt, fic5, history5.txt, forgotte, srex.txt


